In [100]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import CSVLogger, ReduceLROnPlateau

In [101]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import CSVLogger, ReduceLROnPlateau
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.layers import Dense, Flatten, Dropout, UpSampling2D, BatchNormalization, Conv2D, MaxPool2D, InputLayer, GlobalAveragePooling2D

In [102]:
train_df = pd.read_csv('train_frames_20.csv')
test_df = pd.read_csv('test_frames_20.csv')
train_df

,video,frame_path,label
0,GcluCxjiSjI,train/0_0.jpg,tap dancing
1,GcluCxjiSjI,train/0_1.jpg,tap dancing
2,GcluCxjiSjI,train/0_2.jpg,tap dancing
3,GcluCxjiSjI,train/0_3.jpg,tap dancing
4,GcluCxjiSjI,train/0_4.jpg,tap dancing
...,...,...,...
11167,LT-e_wj6d9w,train/617_16.jpg,tango dancing
11168,LT-e_wj6d9w,train/617_17.jpg,tango dancing
11169,LT-e_wj6d9w,train/617_18.jpg,tango dancing
11170,LT-e_wj6d9w,train/617_19.jpg,tango dancing


In [103]:
test_df

,video,frame_path,label
0,BheDJtlAYpA,test/25_0.jpg,tap dancing
1,BheDJtlAYpA,test/25_1.jpg,tap dancing
2,BheDJtlAYpA,test/25_2.jpg,tap dancing
3,BheDJtlAYpA,test/25_3.jpg,tap dancing
4,BheDJtlAYpA,test/25_4.jpg,tap dancing
...,...,...,...
1522,F9Ehijaqdl0,test/587_16.jpg,tango dancing
1523,F9Ehijaqdl0,test/587_17.jpg,tango dancing
1524,F9Ehijaqdl0,test/587_18.jpg,tango dancing
1525,F9Ehijaqdl0,test/587_19.jpg,tango dancing


In [104]:
y_train = train_df["label"]
x_train = train_df.drop(labels = ["label", "video"], axis = 1)

y_test = test_df["label"]
x_test = test_df.drop(labels = ["label", "video"], axis = 1)

In [105]:
label_train_to_int = {label_name: i for i, label_name in enumerate(set(train_df['label']))}
label_test_to_int = {label_name: i for i, label_name in enumerate(set(test_df['label']))}

In [106]:
label_train_to_int

{'square dancing': 0,
 'country line dancing': 1,
 'robot dancing': 2,
 'tap dancing': 3,
 'jumpstyle dancing': 4,
 'mosh pit dancing': 5,
 'breakdancing': 6,
 'belly dancing': 7,
 'dancing macarena': 8,
 'swing dancing': 9,
 'tango dancing': 10,
 'dancing ballet': 11,
 'salsa dancing': 12,
 'dancing gangnam style': 13,
 'dancing charleston': 14}

In [107]:
label_test_to_int

{'square dancing': 0,
 'country line dancing': 1,
 'robot dancing': 2,
 'tap dancing': 3,
 'jumpstyle dancing': 4,
 'mosh pit dancing': 5,
 'breakdancing': 6,
 'belly dancing': 7,
 'dancing macarena': 8,
 'swing dancing': 9,
 'tango dancing': 10,
 'dancing ballet': 11,
 'salsa dancing': 12,
 'dancing gangnam style': 13,
 'dancing charleston': 14}

In [108]:
train_labels = [label_train_to_int[label_name] for label_name in y_train]
test_labels = [label_test_to_int[label_name] for label_name in y_test]

In [109]:
# train_labels = to_categorical(train_labels, num_classes=15)
# train_labels = train_labels.astype('float32')
# test_labels = to_categorical(test_labels, num_classes=15)
# test_labels = test_labels.astype('float32')


In [110]:
test_labels

[3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,


In [111]:
def load_n_preprocess(frame_path):
    image = tf.io.read_file(f'{frame_path}')
    image = tf.image.decode_jpeg(image, channels = 3)
    image = tf.image.resize(image, [128,128])
    image = preprocess_input(image)
    image = image / 225

    return image

In [112]:
def lnp_all_images(df):
    images = [load_n_preprocess(frame_path) for frame_path in df['frame_path']]
    return tf.stack(images)

In [113]:
x_train = lnp_all_images(x_train)
train_frames_dataset = tf.data.Dataset.from_tensor_slices(x_train)

train_labels_tensor = tf.convert_to_tensor(train_labels)
train_labels_dataset = tf.data.Dataset.from_tensor_slices(train_labels_tensor)
train_dataset = tf.data.Dataset.zip((train_frames_dataset, train_labels_dataset))

x_test = lnp_all_images(x_test)
test_frames_dataset = tf.data.Dataset.from_tensor_slices(x_test)

test_labels_tensor = tf.convert_to_tensor(test_labels)
test_labels_dataset = tf.data.Dataset.from_tensor_slices(test_labels_tensor)
test_dataset = tf.data.Dataset.zip((test_frames_dataset, test_labels_dataset))

In [114]:
print(train_frames_dataset.element_spec)

TensorSpec(shape=(128, 128, 3), dtype=tf.float32, name=None)


In [115]:

train_dataset = train_dataset.shuffle(len(train_dataset)).batch(64)


In [116]:
train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [117]:

test_dataset = test_dataset.shuffle(len(test_dataset)).batch(64)


In [118]:
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Замораживание базовой модели
base_model.trainable = False

# Добавление новых слоев для классификации
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(4096, activation='relu',name='fc-1'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096, activation='relu',name='fc-2'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(15, activation='softmax'),
])
model.build(input_shape=(None,128,128,3))
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 4, 4, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc-1 (Dense)                    │ (None, 4096)           │     8,392,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc-2 (Dense)                    │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │        61,455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 48,823,183 (186.25 MB)

 Trainable params: 25,235,471 (96.27 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [119]:
# model = tf.keras.Sequential([])
# model.add(tf.keras.applications.Xception(weights = 'imagenet', include_top=False, input_shape = (128,128,3)))
# for layer in model.layers:
# 	layer.trainable = False
# # add a global spatial average pooling layer
# model.add(GlobalAveragePooling2D())
# # add fully-connected & dropout layers
# model.add(Dense(4096, activation='relu',name='fc-1'))
# model.add(Dropout(0.5))
# model.add(Dense(4096, activation='relu',name='fc-2'))
# model.add(Dropout(0.2))
# # a softmax layer for 8 classes
# num_classes = 15
# model.add(Dense(num_classes, activation='softmax',name='output_layer'))
# model.build(input_shape=(None,128,128,3))
# 
# model.summary()

In [120]:
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()
loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [121]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [122]:
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=3, min_lr=1e-4)

In [123]:
filename = 'supplementary.csv'
csv_logger = CSVLogger(filename)

In [124]:
history = model.fit(
    train_dataset,
    batch_size=64, 
    epochs=10,
    callbacks=[reduce_lr, csv_logger],
    validation_data=test_dataset,
    verbose = 1
)

Epoch 1/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 229s 1s/step - accuracy: 0.0649 - loss: 3.1399 - val_accuracy: 0.0688 - val_loss: 2.7184 - learning_rate: 0.0010
Epoch 2/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 165s 943ms/step - accuracy: 0.0813 - loss: 2.7060 - val_accuracy: 0.0681 - val_loss: 2.7181 - learning_rate: 0.0010
Epoch 3/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 166s 945ms/step - accuracy: 0.0762 - loss: 2.7028 - val_accuracy: 0.0681 - val_loss: 2.7182 - learning_rate: 0.0010
Epoch 4/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 165s 942ms/step - accuracy: 0.0826 - loss: 2.6990 - val_accuracy: 0.0681 - val_loss: 2.7141 - learning_rate: 0.0010
Epoch 5/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 165s 941ms/step - accuracy: 0.0850 - loss: 2.7006 - val_accuracy: 0.0681 - val_loss: 2.7153 - learning_rate: 2.0000e-04
Epoch 6/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 165s 941ms/step - accuracy: 0.0800 - loss: 2.7010 - val_accuracy: 0.0681 - val_loss: 2.7158 - learning_rate: 2.0000e-04
Epoch 7/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 164s 940ms/step - a

In [125]:
loss, acc = model.evaluate(test_dataset, verbose=0)

In [126]:
train_loss = history.history['loss'][-1]
train_accuracy = history.history['accuracy'][-1]
test_loss = history.history['val_loss'][-1]
test_accuracy = history.history['val_accuracy'][-1]
# test_loss = loss
# test_accuracy = acc
print(f"Train loss: {train_loss:.4f}, Train accuracy: {train_accuracy:.4f}")
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}")

Train loss: 2.7003, Train accuracy: 0.0825
Test loss: 2.7162, Test accuracy: 0.0681


In [127]:
# model.save('xception_sparce_32.keras')

In [128]:

image_labels = ['country line dancing',
 'tango dancing',
 'swing dancing',
 'jumpstyle dancing',
 'dancing gangnam style',
 'square dancing',
 'dancing charleston',
 'salsa dancing',
 'mosh pit dancing',
 'robot dancing',
 'tap dancing',
 'breakdancing',
 'belly dancing',
 'dancing ballet',
 'dancing macarena']


In [129]:
def predict_video(video):
    classes = []
    prob = []
    image_names = test_df.loc[test_df['video'] == video, 'frame_path']
    for image_name in image_names:
        image = tf.io.read_file(image_name)
        image = tf.image.decode_jpeg(image, channels = 3)
        image = tf.image.resize(image, [128,128])
        image = np.expand_dims(image, axis=0)
        image = image / 225
        pred = model.predict(image, verbose=0)
        predicted_label_index = np.argmax(pred[0])
        probability = pred[0][predicted_label_index]
        # print(f"Предсказанный класс: {image_labels[predicted_label_index]}, Вероятность: {probability:.4f}")
        classes.append(image_labels[predicted_label_index])
        prob.append(probability)
    result_dict = pd.DataFrame({'label': classes, 'probability': prob})
    max_dict_prob = result_dict.mode()['probability'].max()
    pdict = result_dict[result_dict['label'] == f"{result_dict.mode()['label'][0]}"]
    return max_dict_prob, pdict, result_dict

In [130]:
videos = test_df['video'].unique()
videos

array(['BheDJtlAYpA', '3hII9QR7sag', 'EgM_PTRnV2k', 'iT088BHtz5Y',
       '3gaE6kCFXMg', 'OETUdxEamNI', '8KB7Y262bnE', 'S6oaDdsOVX0',
       '45PO7whfm0A', 'vpWkTE9AbAE', 'UpNvGpDvDiA', 'b0tigV6LTi0',
       'U7qIS5cRhok', 'brE_rDvfn1g', 'bfoFYNHN1po', 'S3IEpNzRdgw',
       'iSIUI8mnU4Y', 'p6Qp9KyqPrM', 'xx4c-7-FNjE', 'IJLpbYXHpbQ',
       'wjL3VY73HqI', '1jNw6TUXpqQ', 'xsRNDo8Upys', 'Qho8YFNeyd8',
       'I6FLGOi43WQ', 'JEMtzhgvH9w', 'og0NG5r04gQ', 'N4_W7KkwaH0',
       'S6PiB8G06ls', 'WCEqNkSK68w', 'ab0f7P0X-uo', 'iFHoIxvt3Lc',
       'QAso9ki6KQ8', '-onTnotvtPI', 'Q5bjuC2zF4s', 'JJcoEwO0yc8',
       '4uCa0PgPRK0', 'a3zZOA0-Q0c', 'QuN5OYQ0K7Q', 'jop0c4PBByQ',
       '7aaP0JOWqto', 'H7BLjO_mVJo', 'Wi5sejYC6xs', 'hM6RQslR6B4',
       'jGuz4i3DYSs', 'WFF2528OhLU', 'uDssN4AhH7g', 'kUvBdvAU_pE',
       'lEvO4vRJgJ0', 'QZ6FBg5EkmA', 'S1IBhxdrK8Y', 'AFOA-p3S0gI',
       'u4crGNsyKuE', 'EfZlpMhU2h8', 'So-qvuOygvY', 'HfOpoduIfCM',
       'uxalPdl8yfw', 'yNNwLGZS9_8', 'b2z51ez006g', '-9N39otwJ

In [131]:
for video in videos:
    max_dict_prob, pdict, result_dict = predict_video(video)
    print("_______________________________________")
    print(f"Видео: {video}")
    print(f"Класс видео: {test_df.loc[test_df['video'] == video, 'label'].values[0]}")
    
    print(f"Наиболее встречающийся класс: {pdict['label'].mode()[0]} , средняя вероятность {pdict['probability'].mean()}")
    if len(pdict.loc[result_dict['probability'] == max_dict_prob, 'label'].values) > 0:
        max_label = pdict.loc[result_dict['probability'] == max_dict_prob, 'label'].values[0]
        print(f"Наибольшая вероятность: {max_dict_prob}, класс {max_label}")
    else:
        print("Не удалось найти соответствующий класс для наибольшей вероятности.")


_______________________________________
Видео: BheDJtlAYpA
Класс видео: tap dancing
Наиболее встречающийся класс: country line dancing , средняя вероятность 0.08252207934856415
Наибольшая вероятность: 0.08257653564214706, класс country line dancing
_______________________________________
Видео: 3hII9QR7sag
Класс видео: tap dancing
Наиболее встречающийся класс: country line dancing , средняя вероятность 0.08247110247612
Наибольшая вероятность: 0.08256381005048752, класс country line dancing
_______________________________________
Видео: EgM_PTRnV2k
Класс видео: tap dancing
Наиболее встречающийся класс: country line dancing , средняя вероятность 0.08261396735906601
Наибольшая вероятность: 0.08269894868135452, класс country line dancing
_______________________________________
Видео: iT088BHtz5Y
Класс видео: tap dancing
Наиболее встречающийся класс: country line dancing , средняя вероятность 0.0826508030295372
Наибольшая вероятность: 0.08283104002475739, класс country line dancing
________

In [132]:
# print(f"Класс видео: {frames_df.loc[frames_df['video'] == video, 'label'].values[0]}")
# print("_______________________________________")
# print("Предсказанные классы по фреймам:")
# print(dict)
# print(f"Наиболее встречающийся класс: {dict.mode()['label'][0]} , средняя вероятность {pdict['probability'].mean()}")
# print(f"Наибольшая вероятность: {max_dict_prob}, класс {dict.loc[dict['probability'] == max_dict_prob, 'label'].values[0]}")

Видео: x0LzgIUDIes
Модель на 20 кадров в видео
Класс видео: mosh pit dancing
_______________________________________
Предсказанные классы по фреймам:
                label  probability
0   jumpstyle dancing     0.153224
1   jumpstyle dancing     0.146594
2   jumpstyle dancing     0.151000
3   jumpstyle dancing     0.141237
4   jumpstyle dancing     0.131609
5   jumpstyle dancing     0.169623
6   jumpstyle dancing     0.172320
7   jumpstyle dancing     0.180115
8       belly dancing     0.124170
9    dancing macarena     0.122473
10   dancing macarena     0.161632
11   dancing macarena     0.131638
12      belly dancing     0.123214
13   mosh pit dancing     0.214120
14   mosh pit dancing     0.205695
15   mosh pit dancing     0.295078
16     square dancing     0.211966
17   mosh pit dancing     0.280752
18   mosh pit dancing     0.335356
19   mosh pit dancing     0.221957
20   mosh pit dancing     0.236604
Наиболее встречающийся класс: jumpstyle dancing , средняя вероятность 0.15571531653404236
Наибольшая вероятность: 0.33535560965538025, класс mosh pit dancing

In [133]:
#TODO: добавить метрики для видео, подача 100 видео, разделение test/train поменять